# News CSV Preprocessing for KB‑ALBERT
This notebook demonstrates a lightweight preprocessing pipeline for Korean stock‑related news data, preparing it for fine‑tuning or inference with the **KB‑ALBERT‑char‑v2** model.

## 1. Load raw CSV files
Place your raw news CSV files in a directory (e.g. `./data/`). All CSVs must share the schema described in your project:

| column | description |
| --- | --- |
| `stock_name` | Stock ticker name |
| `date` | News date `YYYY.MM.DD` |
| `title` | News headline |
| `source` | Publisher name |
| `content` | Article body (plain text) |
| `link` | Original article URL |


In [19]:
import pandas as pd, glob

def load_csvs(path_pattern='/Users/yujimin/KB AI CHALLENGE/project/results/*.csv'):
    files = glob.glob(path_pattern)
    print(f'Found {len(files)} CSV file(s)')
    return pd.concat((pd.read_csv(f, encoding='utf-8') for f in files), ignore_index=True)

raw_df = load_csvs()
print(raw_df.head())

Found 3 CSV file(s)
  stock_name          datetime                                       title  \
0      NAVER  2025.08.05 21:11  [데일리안 오늘뉴스 종합] 이춘석 '주식 차명거래 의혹'…난처한 與 "...   
1      NAVER  2025.08.05 20:41           “낯익은 얼굴” 국가대표됐다더니…‘1400억 잭팟’ 또 대박   
2      NAVER  2025.08.05 20:23  구글 “가림 처리해 보안 우려 해소”…정부, 정밀 지도 반출 이번엔.....   
3      NAVER  2025.08.05 16:08   구글, ‘지도 반출’ 논란에 “흐릿하게 처리된 국내 위성 사진 구매....   
4      NAVER  2025.08.05 14:50         구글, '가림 처리' 국내 위성사진 구매 검토…보안우려 의식했나   

  source                                            content  \
0   데일리안  이춘석 국회 법제사법위원장이 지난 1일 오후 국회에서 열린 법제사법위원회 전체회의 ...   
1  헤럴드경제  김성훈 업스테이지 대표. [업스테이지 유튜브 갈무리][헤럴드경제=권제인 기자] “스...   
2   경향신문  세 차례 1:5000 데이터 요청이번엔 보안 시설 등 흐릿한국내 위성 사진 구매안 ...   
3   경향신문  로이터연합뉴스구글이 정부의 정밀 지도 반출 여부 결정을 앞두고 보안시설 등을 흐릿하...   
4    뉴스1  정부 반출 결정 앞두고 "요구사항 이행 방안 긴밀히 협의 중"구글 "1:5000은 ...   

                                                link  
0  https://finance.naver.com/item/news_read.naver...  
1  https://finance.nav

## 2. Minimal text‑level cleaning
- **Drop rows** with missing `title` or `content`.
- **Remove duplicates** based on `link`.
- **Normalize whitespace & Unicode**, strip URLs and stray special chars.


In [21]:
# 불필요한 노이즈 걷어내고 모델 학습 신호 최대한 보존
import re, unicodedata
import pandas as pd

url_pattern = re.compile(r'https?://\S+')

def normalize(text: str) -> str:
    text = unicodedata.normalize('NFKC', str(text))
    text = url_pattern.sub('', text)          # strip URLs
    text = re.sub(r'\s+', ' ', text).strip() # collapse whitespace
    return text

def preprocess(df: pd.DataFrame) -> pd.DataFrame:
    df = df.dropna(subset=['title', 'content'])
    df = df.drop_duplicates(subset=['link'])
    df['title'] = df['title'].apply(normalize)
    df['content'] = df['content'].apply(normalize)
    # combine title & body for model input
    df['text'] = df['title'] + ' [SEP] ' + df['content']
    # retain only useful cols
    return df[['stock_name', 'datetime', 'text', 'link']]

clean_df = preprocess(raw_df)
print('After cleaning:', clean_df.shape)
clean_df.head()

After cleaning: (598, 4)


,stock_name,datetime,text,link
0,NAVER,2025.08.05 21:11,"[데일리안 오늘뉴스 종합] 이춘석 '주식 차명거래 의혹'...난처한 與 ""... [...",https://finance.naver.com/item/news_read.naver...
1,NAVER,2025.08.05 20:41,“낯익은 얼굴” 국가대표됐다더니...‘1400억 잭팟’ 또 대박 [SEP] 김성훈 ...,https://finance.naver.com/item/news_read.naver...
2,NAVER,2025.08.05 20:23,"구글 “가림 처리해 보안 우려 해소”...정부, 정밀 지도 반출 이번엔..... [...",https://finance.naver.com/item/news_read.naver...
3,NAVER,2025.08.05 16:08,"구글, ‘지도 반출’ 논란에 “흐릿하게 처리된 국내 위성 사진 구매.... [SEP...",https://finance.naver.com/item/news_read.naver...
4,NAVER,2025.08.05 14:50,"구글, '가림 처리' 국내 위성사진 구매 검토...보안우려 의식했나 [SEP] 정부...",https://finance.naver.com/item/news_read.naver...


## 3. Save cleaned corpus
Two common formats:
- **Parquet** (efficient columnar storage)
- **CSV** (for quick inspection)


In [8]:
# clean_df.to_parquet('news_clean.parquet', index=False)
clean_df.to_csv('news_clean.csv', index=False, encoding='utf-8')
print('✅ Saved `news_clean.parquet` and `news_clean.csv`')

✅ Saved `news_clean.parquet` and `news_clean.csv`


In [ ]:
import os
os.getcwd()  # 현재 작업 디렉터리 절대경로 확인

'/Users/yujimin/Downloads'

---
### Next steps
Load `news_clean.parquet` with **Huggingface Datasets** and tokenize using `AlbertTokenizer`:
```python
from datasets import Dataset
from transformers import AlbertTokenizer

ds = Dataset.from_parquet('news_clean.parquet')
tokenizer = AlbertTokenizer.from_pretrained('./kb-albert-char-v2')
def tok(batch):
    return tokenizer(batch['text'], truncation=True, max_length=512)
ds = ds.map(tok, batched=True)
```


In [ ]:
# # 아래 예시는 원본 CSV 묶음(./data/*.csv) 과 정제 결과(news_clean.csv) 를 비교해 
# # “중복 제거·텍스트 정규화가 실제로 어떻게 반영됐는지” 몇 가지 샘플을 바로 확인할 수 있는 
# # 판다스 스크립트입니다.

# import pandas as pd, glob

# # ── 0. 파일 경로 설정 ────────────────────────────────────────────────
# RAW_PATH   = '/Users/yujimin/KB AI CHALLENGE/project/results/*.csv'      # 원본 CSV 모아둔 폴더
# CLEAN_FILE = '/Users/yujimin/KB AI CHALLENGE/project/news_clean.csv'    # 전처리 결과

# # ── 1. 데이터 로드 ───────────────────────────────────────────────────
# orig_df  = pd.concat((pd.read_csv(f, encoding='utf-8') for f in glob.glob(RAW_PATH)),
#                      ignore_index=True)
# clean_df = pd.read_csv(CLEAN_FILE, encoding='utf-8')

# print(f'원본    : {orig_df.shape}  rows')
# print(f'전처리본: {clean_df.shape} rows\n')

# # ── 2. (예시 A) 중복으로 삭제된 기사 살펴보기 ───────────────────────
# dropped_links = set(orig_df['link']) - set(clean_df['link'])
# dropped = orig_df[orig_df['link'].isin(dropped_links)]
# print(f'🗑️  중복/결측으로 제거된 기사 수: {len(dropped)}')
# display(dropped.head(3))

# # ── 3. (예시 B) 제목·본문 정규화 차이 확인 ─────────────────────────
# # clean_df['text'] = "<title> [SEP] <content>" 형태 → 제목만 분리
# tmp = clean_df[['link', 'text']].copy()
# tmp['title_clean'] = tmp['text'].str.split(' [SEP] ').str[0]

# merged = orig_df.merge(tmp[['link', 'title_clean']], on='link', how='inner')

# diff_title = merged[merged['title'] != merged['title_clean']]
# print(f'✏️  정규화로 바뀐 제목 수: {len(diff_title)}')
# display(diff_title[['title', 'title_clean']].head(3))

# # ── 4. (예시 C) 본문 공백·URL 제거 예시 ────────────────────────────
# def show_sample(idx):
#     print("\n원본 본문:")
#     print(orig_df.loc[idx, 'content'][:300], '...')
#     print("\n정제 본문:")
#     print(clean_df.loc[idx, 'text'].split(' [SEP] ')[1][:300], '...')

# # 중복 제거 없는 임의 기사 인덱스 찾아서 시연
# sample_idx = orig_df[~orig_df['link'].isin(dropped_links)].index[0]
# show_sample(sample_idx)


원본    : (8066, 6)  rows
전처리본: (8065, 4) rows

🗑️  중복/결측으로 제거된 기사 수: 1


,stock_name,date,title,source,content,link
249,현대차,2025.08.07,"[속보] 현대차, 미국 GM과 차량 5종 공동 개발한다",디지털타임스,NaN,https://finance.naver.com/item/news_read.naver...


✏️  정규화로 바뀐 제목 수: 8065


,title,title_clean
0,"HD현대, 美 안두릴과 손잡고 AI 무인군함 개발한다","HD현대, 美 안두릴과 손잡고 AI 무인군함 개발한다 [SEP] 연합뉴스HD현대가 ..."
1,'베트남 서열 1위' 방한…李대통령 만찬에 재계 총수들 참석,'베트남 서열 1위' 방한...李대통령 만찬에 재계 총수들 참석 [SEP] [서울=...
2,[단독] '베트남 서열 1위' 방한 만찬에 대기업 총수들 참석,[단독] '베트남 서열 1위' 방한 만찬에 대기업 총수들 참석 [SEP] 10일 방...



원본 본문:
연합뉴스HD현대가 미국의 인공지능(AI) 방산기업 안두릴 인터스트리와 손잡고 함정 개발에 나선다.  AI 함정 기술을 함께 개발해 한미 양국 시장에 진출하겠다는 계획이다.HD현대는 안두릴과 경기도 성남 HD현대 글로벌R&amp;D센터(GRC)에서 ‘함정 개발 협력을 위한 합의각서(MOA)’를 체결했다. 앞서 HD현대와 안두릴은 지난 4월 함정 개발 협력을위한  업무협약(MOU)를 맺었는데 이번 MOA는 협력 내용을 더 구체화한 협약이다. 양사는 이번 MOA를 통해 HD현대는 AI 함정 자율화 기술 및 함정 설계·기술을 제공하고 안두 ...

정제 본문:
연합뉴스HD현대가 미국의 인공지능(AI) 방산기업 안두릴 인터스트리와 손잡고 함정 개발에 나선다. AI 함정 기술을 함께 개발해 한미 양국 시장에 진출하겠다는 계획이다.HD현대는 안두릴과 경기도 성남 HD현대 글로벌R&amp;D센터(GRC)에서 ‘함정 개발 협력을 위한 합의각서(MOA)’를 체결했다. 앞서 HD현대와 안두릴은 지난 4월 함정 개발 협력을위한 업무협약(MOU)를 맺었는데 이번 MOA는 협력 내용을 더 구체화한 협약이다. 양사는 이번 MOA를 통해 HD현대는 AI 함정 자율화 기술 및 함정 설계·기술을 제공하고 안두릴은 ...


In [ ]:
# import pandas as pd, glob, re, html, unicodedata

# # ── 0. 파일 경로 설정 ────────────────────────────────────────────────
# RAW_PATH   = '/Users/yujimin/KB AI CHALLENGE/project/results/*.csv'      # 원본 CSV 모아둔 폴더
# CLEAN_FILE = '/Users/yujimin/KB AI CHALLENGE/project/news_clean.csv'    # 전처리 결과

# orig_df  = pd.concat((pd.read_csv(f) for f in glob.glob(RAW_PATH)), ignore_index=True)
# clean_df = pd.read_csv(CLEAN_FILE)

# # ── 1) link 기준 inner-join
# merged = orig_df[['link', 'title']].merge(clean_df[['link', 'text']], on='link', how='inner')

# # ── 2) 제목 부분만 추출 ─────────────────────────────────────────────
# sep_regex = re.compile(r'\s*\[\s*SEP\s*\]\s*', flags=re.IGNORECASE)
# merged['title_clean_raw'] = merged['text'].str.split(sep_regex).str[0]

# # ── 3) 동일 규칙으로 양쪽 정규화
# def normalize(t: str) -> str:
#     t = unicodedata.normalize('NFKC', str(t))
#     t = html.unescape(t)                # &amp; → &
#     t = re.sub(r'https?://\S+', '', t)  # URL 제거
#     t = re.sub(r'\s+', ' ', t).strip()  # 공백 축소
#     return t

# merged['title_norm_orig']  = merged['title'].apply(normalize)
# merged['title_norm_clean'] = merged['title_clean_raw'].apply(normalize)

# # ── 4) 일치율 재확인
# match_rate = (merged['title_norm_orig'] == merged['title_norm_clean']).mean()
# print(f"🔍 정규화 후 제목 완전 일치율: {match_rate:.1%}")

# # 불일치 샘플 3건만 확인
# mismatch = merged.query('title_norm_orig != title_norm_clean').head(3)
# display(mismatch[['title_norm_orig', 'title_norm_clean']])



🔍 정규화 후 제목 완전 일치율: 100.0%


,title_norm_orig,title_norm_clean


In [ ]:
# # clean_df 내부 ‘중복 행’ 탐색 스니펫
# # ───────────────────────────────────────────
# import pandas as pd

# # (가정) 이미 clean_df DataFrame 이 메모리에 존재
# CLEAN_FILE = '/Users/yujimin/KB AI CHALLENGE/project/news_clean.csv'  

# # 1) link 기준 ― 가장 확실한 중복 체크
# dup_by_link = clean_df[clean_df.duplicated(subset=["link"], keep=False)]
# print(f"[link] 기준 중복 건수  : {dup_by_link.shape[0]}")
# # display(dup_by_link.head(3))

# # 2) title+date 기준 ― 동일 기사인데 링크가 바뀌었을 가능성 탐지
# #    text 컬럼에서 제목만 분리해 임시 컬럼 생성
# clean_df["title"] = clean_df["text"].str.split(r"\s*\[\s*SEP\s*\]\s*", n=1).str[0]

# dup_by_title_date = clean_df[
#     clean_df.duplicated(subset=["title", "date"], keep=False)
# ]
# print(f"[title+date] 기준 중복 건수: {dup_by_title_date.shape[0]}")
# # display(dup_by_title_date.head(3))

# # 3) 전체 컬럼 완전 동일 행
# dup_full = clean_df[clean_df.duplicated(keep=False)]
# print(f"[전체 컬럼] 완전 중복 행수 : {dup_full.shape[0]}")
# # display(dup_full.head(3))


[link] 기준 중복 건수  : 0
[title+date] 기준 중복 건수: 2472
[전체 컬럼] 완전 중복 행수 : 0


,stock_name,date,text,link,title


현황 진단
- link 기준 중복 0건 → URL은 모두 고유

- title + date 기준 중복 2 472건 → 동일한 제목이 같은 날에 여러 URL(언론사·모바일/PC 버전 등)로 존재

- 전체 8 065건 중 약 31 %가 “사실상 같은 기사” 이므로, 모델 학습·추론 효율을 위해 제거를 권장합니다.

In [14]:
#  본문 길이가 가장 긴 기사	전문(全文)·모바일/PC 통합 버전일 가능성 ↑

import pandas as pd, re

CLEAN_FILE = '/Users/yujimin/KB AI CHALLENGE/project/news_clean.csv'  

# 1) 제목 분리해 임시 저장
sep_regex = re.compile(r"\s*\[\s*SEP\s*\]\s*", flags=re.IGNORECASE)
clean_df["title"]   = clean_df["text"].str.split(sep_regex, n=1).str[0]
clean_df["content"] = clean_df["text"].str.split(sep_regex, n=1).str[1]

# 2) 본문 길이 계산
clean_df["len"] = clean_df["content"].str.len()

# 3) len 내림차순 → 중복(title, date) 제거 → 정렬 복원
dedup = (
    clean_df
    .sort_values("len", ascending=False)            # 길이 긴 기사 우선
    .drop_duplicates(subset=["title", "date"], keep="first")
    .sort_index()                                  # 원래 순서로 정렬(선택)
    .drop(columns=["title", "content", "len"])     # 임시 컬럼 정리
)

print(f"⚙️  중복 제거 전: {clean_df.shape[0]:,}")
print(f"⚙️  중복 제거 후: {dedup.shape[0]:,}")

# 4) 저장
dedup.to_csv("news_clean_dedup.csv", index=False, encoding="utf-8")
print("✅ Saved → news_clean_dedup.csv")


⚙️  중복 제거 전: 8,065
⚙️  중복 제거 후: 6,716
✅ Saved → news_clean_dedup.csv


In [15]:
clean_df.head()

,stock_name,date,text,link,title,content,len
0,현대차,2025.08.07,"HD현대, 美 안두릴과 손잡고 AI 무인군함 개발한다 [SEP] 연합뉴스HD현대가 ...",https://finance.naver.com/item/news_read.naver...,"HD현대, 美 안두릴과 손잡고 AI 무인군함 개발한다",연합뉴스HD현대가 미국의 인공지능(AI) 방산기업 안두릴 인터스트리와 손잡고 함정 ...,1200
1,현대차,2025.08.07,'베트남 서열 1위' 방한...李대통령 만찬에 재계 총수들 참석 [SEP] [서울=...,https://finance.naver.com/item/news_read.naver...,'베트남 서열 1위' 방한...李대통령 만찬에 재계 총수들 참석,[서울=뉴시스] 최진석 기자 = 이재명 대통령이 13일 서울 용산 대통령실에서 열린...,841
2,현대차,2025.08.07,[단독] '베트남 서열 1위' 방한 만찬에 대기업 총수들 참석 [SEP] 10일 방...,https://finance.naver.com/item/news_read.naver...,[단독] '베트남 서열 1위' 방한 만찬에 대기업 총수들 참석,10일 방한하는 또 럼 베트남 공산당 서기장이 이틀에 걸쳐 국내 주요 대기업 총수 ...,1705
3,현대차,2025.08.07,"소비쿠폰 먹고 마시는 데 주로 썼다...소상공인 매출증대로 [SEP] 음식점, 마트...",https://finance.naver.com/item/news_read.naver...,소비쿠폰 먹고 마시는 데 주로 썼다...소상공인 매출증대로,"음식점, 마트·식료품, 병원·약국 등 매츨 늘어나2주간 5조7679억원 지금···2...",1274
4,현대차,2025.08.07,"밥 먹고, 병원비 내고.. '소비쿠폰' 2조 원 어디 몰렸나 봤더니 [SEP] 제주...",https://finance.naver.com/item/news_read.naver...,"밥 먹고, 병원비 내고.. '소비쿠폰' 2조 원 어디 몰렸나 봤더니",제주의 한 민생회복 소비쿠폰 사용 가능 매장전 국민에게 지급된 민생회복 소비쿠폰이 ...,1402


In [18]:
import pandas as pd

# ── 0) CSV 로드 ────────────────────────────
df = pd.read_csv("/Users/yujimin/KB AI CHALLENGE/project/news_clean_dedup.csv", encoding="utf-8")

# ── 1) date 컬럼을 날짜형으로 변환 ────────
df["date"] = pd.to_datetime(df["date"], format="%Y.%m.%d")

# ── 2) 종목별 날짜 범위 요약 ──────────────
range_tbl = (
    df.groupby("stock_name")["date"]
      .agg(start="min", end="max", days="nunique", articles="count")
      .reset_index()
      .sort_values("stock_name")
)
print("📅 종목별 날짜 범위·기사 수")
print(range_tbl.to_string(index=False))

# 1) 종목-날짜별 기사 수 집계
cnt = (
    df.groupby(["stock_name", "date"])
      .size()
      .reset_index(name="articles")
)

# 2) 종목별 상위 5일 추출
top5_each = (
    cnt.sort_values(["stock_name", "articles"], ascending=[True, False])
       .groupby("stock_name")
       .head(5)
)

# 3) 보기 좋게 출력
for stock, sub in top5_each.groupby("stock_name"):
    print(f"\n📰 {stock} – 기사 수 상위 5일")
    print(sub.sort_values("articles", ascending=False)
              .reset_index(drop=True)
              .to_string(index=False))


📅 종목별 날짜 범위·기사 수
stock_name      start        end  days  articles
     NAVER 2025-07-22 2025-08-07    17      1664
      삼성전자 2025-07-31 2025-08-07     8      1680
       카카오 2025-07-07 2025-08-07    32      1710
       현대차 2025-07-28 2025-08-07    11      1662

📰 NAVER – 기사 수 상위 5일
stock_name       date  articles
     NAVER 2025-08-04       238
     NAVER 2025-08-05       206
     NAVER 2025-07-23       161
     NAVER 2025-07-24       161
     NAVER 2025-07-30       124

📰 삼성전자 – 기사 수 상위 5일
stock_name       date  articles
      삼성전자 2025-07-31       286
      삼성전자 2025-08-07       285
      삼성전자 2025-08-01       257
      삼성전자 2025-08-05       251
      삼성전자 2025-08-06       222

📰 카카오 – 기사 수 상위 5일
stock_name       date  articles
       카카오 2025-08-07       151
       카카오 2025-08-05       116
       카카오 2025-07-21       103
       카카오 2025-07-14       101
       카카오 2025-07-09        88

📰 현대차 – 기사 수 상위 5일
stock_name       date  articles
       현대차 2025-07-31       314
       현대차 2025

In [23]:
# -*- coding: utf-8 -*-
"""
Naver Finance 뉴스 → 통합·정제·중복 제거 → news_clean_YYYYMMDD.csv
"""
import glob, re, unicodedata, pandas as pd
from collections import Counter

# ───────────────────────────────
# 1. 파일 로드
# ───────────────────────────────
def load_csvs(pattern="/Users/yujimin/KB AI CHALLENGE/project/results/*.csv") -> pd.DataFrame:
    files = glob.glob(pattern)
    print(f"📂 Found {len(files)} CSV file(s)")
    return pd.concat((pd.read_csv(f, encoding="utf-8") for f in files), ignore_index=True)

# ───────────────────────────────
# 2. 텍스트 정규화
# ───────────────────────────────
URL_RE = re.compile(r"https?://\S+")

def normalize(text: str) -> str:
    text = unicodedata.normalize("NFKC", str(text))
    text = URL_RE.sub("", text)
    return re.sub(r"\s+", " ", text).strip()

# ───────────────────────────────
# 3. 전처리 & 중복 제거
# ───────────────────────────────
def preprocess(df: pd.DataFrame) -> pd.DataFrame:
    df = df.dropna(subset=["title", "content"]).drop_duplicates(subset=["link"])

    df["title"]   = df["title"].apply(normalize)
    df["content"] = df["content"].apply(normalize)
    df["text"]    = df["title"] + " [SEP] " + df["content"]

    # (선택) 동일 제목·날짜 중복 제거 – 본문 길이 긴 기사 우선
    df["len"] = df["content"].str.len()
    df = (
        df.sort_values("len", ascending=False)
          .drop_duplicates(subset=["title", "datetime"], keep="first")
          .drop(columns="len")
    )
    return df[["stock_name", "datetime", "text", "link"]]

# ───────────────────────────────
# 4. 저장 파일명 결정 로직
# ───────────────────────────────
def decide_filename(df: pd.DataFrame, out_dir="/Users/yujimin/KB AI CHALLENGE/project") -> str:
    # datetime 컬럼에서 ‘YYYY.MM.DD’ 부분만 추출
    dates = df["datetime"].astype(str).str.extract(r"(\d{4}\.\d{2}\.\d{2})")[0]
    # 가장 이른 날짜(earliest) 선택 — 필요 시 latest/most common 으로 교체
    target = pd.to_datetime(dates, format="%Y.%m.%d").min().strftime("%Y%m%d")
    return f"{out_dir}/news_clean_{target}.csv"

# ───────────────────────────────
# 5. 메인
# ───────────────────────────────
if __name__ == "__main__":
    raw_df   = load_csvs()
    clean_df = preprocess(raw_df)
    print(f"✅ After cleaning: {clean_df.shape}")

    out_csv = decide_filename(clean_df)
    clean_df.to_csv(out_csv, index=False, encoding="utf-8")
    print(f"🎉 Saved → {out_csv}")


📂 Found 4 CSV file(s)
✅ After cleaning: (728, 4)
🎉 Saved → /Users/yujimin/KB AI CHALLENGE/project/news_clean_20250805.csv
